In [73]:
import torch


In [74]:
from vit import VisionTransformer

In [75]:
model = VisionTransformer((64,48),in_chans=256,embed_dim=2304)

In [76]:
input = torch.rand([32,256,64,64])

In [77]:
model(input).shape

AssertionError: Input image size (64*64) doesn't match model (64*48).